## Download required data from yahoo finance

### Import libraries

In [5]:
import yfinance as yf
import pandas as pd

In [6]:
import asyncio
from twscrape import API, gather

### Download the stock data

In [11]:
# NVDA is the stock ticker of Nvidia on the NYSE
ticker = "NVDA"
start_date = "2024-01-01"
end_date = "2024-03-31"
# download the data from Yahoo Finance
data = yf.download(ticker, start=start_date, end=end_date) 
# convert the downloaded data into csv files
data.to_csv(f"stock_data/{ticker}_{start_date[-5:]}-{end_date[-5:]}")
print("Data added to stock_data folder")

[*********************100%%**********************]  1 of 1 completed

Data added to stock_data folder


## twitter data

In [4]:
api = API()

In [5]:
async def scrape_tweets(company, start_date, end_date):
    
    data = []  #create an empty list to be used to store the search results
    
    #define the search query. Include start date and end date
    q = f"${company} until:{end_date} since:{start_date} lang:en" 
    save_to_file = f"twitter_data/{company}_{start_date}-{end_date}"
    
    async for tweet in api.search(q, limit=300000): #iterate over the search results
        c = [tweet.id, tweet.date, tweet.rawContent, tweet.likeCount, tweet.retweetCount, tweet.user.location] #list of attributes to return 
        data.append(c)  #add each new list of attributes to 'data'
    
    df = pd.DataFrame(data, columns=['Tweet_ID', 'Time_Created', 'Text', 'Likes', 'Retweets', 'Location']) #convert the list to a dataframe
    df.to_csv(save_to_file, index = False) #save to a chosen directory on the computer
    return df

In [44]:
nvidia_tweets = await scrape_tweets("NVDA", "2024-01-01", "2024-01-02")
nvidia_tweets

2024-04-23 22:45:58.312 | WARNING  | twscrape.queue_client:_check_rep:162 - Session expired or banned: 401 - -1/-1 - anshul904 - (32) Could not authenticate you
2024-04-23 22:46:40.410 | WARNING  | twscrape.queue_client:_check_rep:162 - Session expired or banned: 401 - -1/-1 - PranayRaturi - (32) Could not authenticate you


,Tweet_ID,Time_Created,Text,Likes,Retweets,Location
0,1741971821996707934,2024-01-01 23:57:01+00:00,"$BTC Hit 44,150\n\nRemember: The ONLY portfoli...",1,0,"Plan the Trade, Trade the Plan"
1,1741965702700097931,2024-01-01 23:32:42+00:00,$BTC I gave Profit Target before price got the...,4,1,"Plan the Trade, Trade the Plan"
2,1741971250354721272,2024-01-01 23:54:45+00:00,@zerohedge @zerohedge \n$META $GOOGL $AMZN $M...,0,0,Borderless
3,1741971091327709350,2024-01-01 23:54:07+00:00,$MULN will PARABOLIC next week 10000X incoming...,0,0,Milky Way
4,1741388323216658564,2023-12-31 09:18:24+00:00,"@Tony_Denaro Tony, what’s your thoughts on MUL...",2,0,
...,...,...,...,...,...,...
635,1741611770790428822,2024-01-01 00:06:18+00:00,$SPY Study Material I just put together for yo...,4,1,"Plan the Trade, Trade the Plan"
636,1741611370309963847,2024-01-01 00:04:42+00:00,"“Most winning trading community, Get next winn...",0,0,
637,1741610940041482422,2024-01-01 00:03:00+00:00,"#NewYear 🎄\nI wish everyone a happy, healthy a...",3,2,"San Francisco, CA"
638,1741610893933441225,2024-01-01 00:02:49+00:00,ALWAYS zoom out the chart to see what’s really...,0,0,
